# Puntualidad en las aerolíneas y factores de influencia
## CML implementado en C

In [ ]:
!cd .. && git submodule init
!cd .. && git submodule update
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

In [ ]:
#Librería de C++ que resuelve Cuadrados Mínimos Lineales, y además provee RMSE y NRMSE
import CML
#Para crear matrices e interfacear con Eigen en C++
import numpy as np
#Para hacer los gráficos
import matplotlib.pyplot as plt
#Para crear dataframes y almacenar los datos.
import pandas as pd
#Para llamar a comandos de la terminal
import os
#Para operar sobre fechas
import datetime as dt
#Para explorar directorios
import glob
#Para cargar los datos desde archivos de texto
import pickle
#¿Entraste con un entorno virtual?
print('Directorio en el que se está ejecutando este notebook:')
!pwd
print(' \nVersión de Python detectada:')
!python --version

In [ ]:
#Descargar los archivos de datos que quieran
URLprefijo = 'http://stat-computing.org/dataexpo/2009/'
archivosDatos = ['1990','1991','1992','1993','1994','1995','1996','2001','2002','2003','2004','2005','2006','2007','2008','airports','carriers','plane-data']

#Descomentar estas dos líneas si quieren descargar todos los datos
#archivosDatos = [str(año) for año in range(1987,2009)]
#archivosDatos.extend(['airports','carriers','plane-data'])

archivosFaltantes = [archivo for archivo in archivosDatos if not glob.glob('../data/' + archivo + '.*')]

print("Archivos de datos a descargar:\n",archivosFaltantes)

with open('../data/dataset_list.txt', 'w') as f:
    for archivo in archivosFaltantes:
        if archivo in ['airports','carriers','plane-data']:
            ext = '.csv'
        else:
            ext = '.csv.bz2'
        f.write(URLprefijo + archivo + ext + '\n')
print("Descargando " + str(len(archivosFaltantes)) + " archivo(s)...")
os.system("cd ../data/ && wget -i dataset_list.txt")

In [ ]:
#Descomprimir todos los archivos de datos, borrar los archivos *.bz2 originales. Hace falta tener bzip2 (probablemente ya lo tengan).
os.system("cd ../data/ && bunzip2 *.bz2 && rm *.bz2")
#Descomprimir los archivos que contienen los datos de OTP.
os.system("cd ../notebooks/ && tar -xvf *tgz && rm *.tgz")

In [ ]:
%%time
#Leo archivos .csv
#archivos = ['1990','1991','1992','1993','1994','1995','1996','2001','2002','2003','2004','2005','2006','2007','2008']
archivos = ['1990','1991','1992','1993','1994','1995','1996']

for año in archivos:
    print("Año",año)
    #Agrego la columna 'Día del año' al dataframe
    df = pd.read_csv('../data/'+año+'.csv', encoding='latin-1')
    df['Day']=df['DayofMonth']
    A = pd.to_datetime(df[['Year','Month','Day']])
    df['Day']=A.dt.dayofyear

    conDelay = df[(df["DepDelay"]>=15) | (df["ArrDelay"]>=15)]
    
    horarios = [100*i for i in range(1,25)]
    horarios.insert(0,1)
    otpPorHorarioPartida = []
    for hhmm in range(len(horarios)-1):
        total = df[(df["CRSDepTime"]>=horarios[hhmm]) & (df["CRSDepTime"]<horarios[hhmm+1])].shape[0]
        otp = conDelay[(conDelay["CRSDepTime"]>=horarios[hhmm]) & (conDelay["CRSDepTime"]<horarios[hhmm+1])].shape[0]/total if total else 0
        otpPorHorarioPartida.append(otp)
    otpPorDia = []
    for dia in range(1,366):
        otp = conDelay[(conDelay["Day"]==dia)].shape[0]/df[(df["Day"]==dia)].shape[0]
        otpPorDia.append(otp)
    otpPorDoW = []
    for dayOfWeek in range(1,8):
        otp = conDelay[(conDelay["DayOfWeek"]==dayOfWeek)].shape[0]/df[(df["DayOfWeek"]==dayOfWeek)].shape[0]
        otpPorDoW.append(otp)
    otpPorMes = []
    for mes in range(1,13):
        otp = conDelay[(conDelay["Month"]==mes)].shape[0]/df[(df["Month"]==mes)].shape[0]
        otpPorMes.append(otp)
    cantPorHorarioPartida = []
    for hhmm in range(len(horarios)-1):
        cant = df[(df["CRSDepTime"]>=horarios[hhmm]) & (df["CRSDepTime"]<horarios[hhmm+1])].shape[0]
        cantPorHorarioPartida.append(cant)
    cantPorDia = []
    for dia in range(1,366):
        cant = df[(df["Day"]==dia)].shape[0]
        cantPorDia.append(cant)
    cantPorDoW = []
    for dayOfWeek in range(1,8):
        cant = df[df["DayOfWeek"]==dayOfWeek].shape[0]
        cantPorDoW.append(cant)
    cantPorMes = []
    for mes in range(1,13):
        cant = df[df["Month"]==mes].shape[0]
        cantPorMes.append(cant)
    cantPorDiaYOrig = []
    with open('otpPorHorarioPartida'+año+'.txt', 'wb') as f:
        pickle.dump(otpPorHorarioPartida, f)
    with open('otpPorDia'+año+'.txt', 'wb') as f:
        pickle.dump(otpPorDia, f)
    with open('otpPorDoW'+año+'.txt', 'wb') as f:
        pickle.dump(otpPorDoW, f)
    with open('otpPorMes'+año+'.txt', 'wb') as f:
        pickle.dump(otpPorMes, f)
    with open('cantPorHorarioPartida'+año+'.txt', 'wb') as f:
        pickle.dump(cantPorHorarioPartida, f)
    with open('cantPorDia'+año+'.txt', 'wb') as f:
        pickle.dump(cantPorDia, f)
    with open('cantPorDoW'+año+'.txt', 'wb') as f:
        pickle.dump(cantPorDoW, f)
    with open('cantPorMes'+año+'.txt', 'wb') as f:
        pickle.dump(cantPorMes, f)